# 1) Loading and cleaning the data

In [3]:
import pandas as pd

In [29]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

#print('Before dropping Boroughs ='.join(df.shape))
df_wikipedia= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

#there are no Postal codes repeated so we dont need to concatinate the Neighbourhoods.

df_wikipedia.shape

(103, 3)

# 2) Loading LAtitude and logitude

In [5]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 6.6MB/s ta 0:00:011


In [26]:
!wget -q -O 'zipcode.csv' http://cocl.us/Geospatial_data

In [30]:
#df.head(1)
#geocoder.google('M3A, Toronto, Ontario')

df1_zip = pd.read_csv('zipcode.csv')
df1_zip.head()

df_merged = pd.merge(df_wikipedia,df1_zip, on=['Postal Code'])

df_merged.head()

#geocoder.google('{}, Toronto, Ontario'.format(df['Postal Code'].iloc(0)))

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
